#### Project-1

## Estimation of **number of claims** using **negative binomial distribution**.

### Importing the required libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from functools import reduce
from scipy import stats
from patsy import dmatrices

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Mounting the google drive to colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = 'drive/My Drive/datasets'

### Importing the datasets for the project.

In [ ]:
# CONTRACTS <- read.csv("/content/CONTRACTS.csv")           # R code
# head(CONTRACTS)

contracts = pd.read_csv(path+'/CONTRACTS.csv')              # Python code
contracts.head()

,Unnamed: 0,PolicyID,ClaimNb,Exposure,Power,CarAge,DriverAge,Brand,Gas,Region,Density
0,1,1,0,0.09,g,0,46,Japanese (except Nissan) or Korean,Diesel,Aquitaine,76
1,2,2,0,0.84,g,0,46,Japanese (except Nissan) or Korean,Diesel,Aquitaine,76
2,3,3,0,0.52,f,2,38,Japanese (except Nissan) or Korean,Regular,Nord-Pas-de-Calais,3003
3,4,4,0,0.45,f,2,38,Japanese (except Nissan) or Korean,Regular,Nord-Pas-de-Calais,3003
4,5,5,0,0.15,g,0,41,Japanese (except Nissan) or Korean,Diesel,Pays-de-la-Loire,60


In [ ]:
pd.DataFrame(contracts.Brand.unique())     # value names are not as same as textbook

,0
0,Japanese (except Nissan) or Korean
1,Fiat
2,"Opel, General Motors or Ford"
3,"Mercedes, Chrysler or BMW"
4,"Renault, Nissan or Citroen"
5,"Volkswagen, Audi, Skoda or Seat"
6,other


In [ ]:
pd.DataFrame(contracts.ClaimNb.unique())

,0
0,0
1,1
2,2
3,3
4,4


In [ ]:
pd.DataFrame(contracts.Power.value_counts())

,Power
f,95718
g,91198
e,77022
d,68014
h,26698
j,18038
i,17616
k,9537
l,4681
m,1832


In [ ]:
pd.DataFrame(contracts['Brand'].value_counts())

,Brand
"Renault, Nissan or Citroen",218200
Japanese (except Nissan) or Korean,79060
"Opel, General Motors or Ford",37402
"Volkswagen, Audi, Skoda or Seat",32638
"Mercedes, Chrysler or BMW",19280
Fiat,16723
other,9866


In [ ]:
# names(CONTRACTS)           # R code

contracts.columns            # Python code

Index(['Unnamed: 0', 'PolicyID', 'ClaimNb', 'Exposure', 'Power', 'CarAge',
       'DriverAge', 'Brand', 'Gas', 'Region', 'Density'],
      dtype='object')

In [ ]:
contracts.shape

(413169, 11)

In [ ]:
contracts.drop('Unnamed: 0',axis = 'columns',inplace = True)
contracts= contracts.reset_index(drop=True)
contracts.sort_values('PolicyID' , ascending=True, inplace=True)

In [ ]:
contracts.shape

(413169, 10)

In [ ]:
contracts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413169 entries, 0 to 413168
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   PolicyID   413169 non-null  int64  
 1   ClaimNb    413169 non-null  int64  
 2   Exposure   413169 non-null  float64
 3   Power      413169 non-null  object 
 4   CarAge     413169 non-null  int64  
 5   DriverAge  413169 non-null  int64  
 6   Brand      413169 non-null  object 
 7   Gas        413169 non-null  object 
 8   Region     413169 non-null  object 
 9   Density    413169 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 34.7+ MB


in the above list **Power, Brand , Gas , Region** are categorical variables.

In [ ]:
contracts.describe()

,PolicyID,ClaimNb,Exposure,CarAge,DriverAge,Density
count,413169.000000,413169.000000,413169.000000,413169.000000,413169.000000,413169.000000
mean,206585.000000,0.039163,0.561088,7.532404,45.319876,1985.153913
std,119271.761023,0.204053,0.369477,5.763011,14.328120,4776.254543
min,1.000000,0.000000,0.002732,0.000000,18.000000,2.000000
25%,103293.000000,0.000000,0.200000,3.000000,34.000000,67.000000
50%,206585.000000,0.000000,0.540000,7.000000,44.000000,287.000000
75%,309877.000000,0.000000,1.000000,12.000000,54.000000,1410.000000
max,413169.000000,4.000000,1.990000,100.000000,99.000000,27000.000000


Continous variables modified as levels (categorized) in R code.

In [ ]:
contracts.isnull().sum()             # no null values

PolicyID     0
ClaimNb      0
Exposure     0
Power        0
CarAge       0
DriverAge    0
Brand        0
Gas          0
Region       0
Density      0
dtype: int64

In [ ]:
contracts.DriverAge = pd.cut(contracts.DriverAge , bins=[17,22,26,42,74,np.inf],labels =None , retbins = None , duplicates = 'raise' , ordered=True , include_lowest=False)

contracts.DriverAge.unique()

[(42.0, 74.0], (26.0, 42.0], (22.0, 26.0], (17.0, 22.0], (74.0, inf]]
Categories (5, interval[float64, right]): [(17.0, 22.0] < (22.0, 26.0] < (26.0, 42.0] <
                                           (42.0, 74.0] < (74.0, inf]]

In [ ]:
contracts.DriverAge.dtype

CategoricalDtype(categories=[(17.0, 22.0], (22.0, 26.0], (26.0, 42.0], (42.0, 74.0], (74.0, inf]], ordered=True)

In [ ]:
contracts.CarAge = pd.cut(contracts.CarAge , bins = [0,15,np.inf] ,labels =None , retbins = None , duplicates = 'raise' , ordered=True , include_lowest=True)

contracts.CarAge.unique()

[(-0.001, 15.0], (15.0, inf]]
Categories (2, interval[float64, right]): [(-0.001, 15.0] < (15.0, inf]]

In [ ]:
contracts.CarAge.dtype

CategoricalDtype(categories=[(-0.001, 15.0], (15.0, inf]], ordered=True)

In [ ]:
contracts.Density = pd.cut(contracts.Density , bins=[0,40,200,500,4500,np.inf] ,labels =None , retbins = None , duplicates = 'raise' , ordered=True , include_lowest=True)

contracts.Density.unique()

[(40.0, 200.0], (500.0, 4500.0], (4500.0, inf], (-0.001, 40.0], (200.0, 500.0]]
Categories (5, interval[float64, right]): [(-0.001, 40.0] < (40.0, 200.0] < (200.0, 500.0] <
                                           (500.0, 4500.0] < (4500.0, inf]]

In [ ]:
# CONTRACTS.f$brandF <- factor(CONTRACTS.f$BRAND=="F",labels=c("other","F"))                              # R-code

contracts['Brand'] = contracts['Brand'].replace("Japanese (except Nissan) or Korean" , "F")
contracts['Brand'] = contracts['Brand'].replace("Renault, Nissan or Citroen" , "other")
contracts['Brand'] = contracts['Brand'].replace("Opel, General Motors or Ford" , "other")                     # Python Code
contracts['Brand'] = contracts['Brand'].replace("Volkswagen, Audi, Skoda or Seat" , "other")
contracts['Brand'] = contracts['Brand'].replace("Mercedes, Chrysler or BMW" , "other")
contracts['Brand'] = contracts['Brand'].replace("Fiat" , "other")
contracts['Brand'] = contracts['Brand'].replace("other" , "other")

In [ ]:
contracts.Brand.unique()

array(['F', 'other'], dtype=object)

In [ ]:
# R-Code
# CONTRACTS.f$powerF <- factor(1*(CONTRACTS.f$POWER%in%letters[4:6]) + 2*(CONTRACTS.f$POWER%in%letters[7:8]) + labels=c("other","DEF","GH"))

contracts['Power'] = contracts['Power'].replace("d" , "def")
contracts['Power'] = contracts['Power'].replace("e" , "def")
contracts['Power'] = contracts['Power'].replace("f" , "def")
contracts['Power'] = contracts['Power'].replace("g" , "gh")
contracts['Power'] = contracts['Power'].replace("h" , "gh")
contracts['Power'] = contracts['Power'].replace("j" , "other")                                # Python Code
contracts['Power'] = contracts['Power'].replace("i" , "other")
contracts['Power'] = contracts['Power'].replace("k" , "other")
contracts['Power'] = contracts['Power'].replace("l" , "other")
contracts['Power'] = contracts['Power'].replace("m" , "other")
contracts['Power'] = contracts['Power'].replace("o" , "other")
contracts['Power'] = contracts['Power'].replace("n" , "other")

In [ ]:
contracts.Power.unique()

array(['gh', 'def', 'other'], dtype=object)

In [ ]:
contracts.Brand.unique()

array(['F', 'other'], dtype=object)

In [ ]:
contracts.head()

,PolicyID,ClaimNb,Exposure,Power,CarAge,DriverAge,Brand,Gas,Region,Density
0,1,0,0.09,gh,"(-0.001, 15.0]","(42.0, 74.0]",F,Diesel,Aquitaine,"(40.0, 200.0]"
1,2,0,0.84,gh,"(-0.001, 15.0]","(42.0, 74.0]",F,Diesel,Aquitaine,"(40.0, 200.0]"
2,3,0,0.52,def,"(-0.001, 15.0]","(26.0, 42.0]",F,Regular,Nord-Pas-de-Calais,"(500.0, 4500.0]"
3,4,0,0.45,def,"(-0.001, 15.0]","(26.0, 42.0]",F,Regular,Nord-Pas-de-Calais,"(500.0, 4500.0]"
4,5,0,0.15,gh,"(-0.001, 15.0]","(26.0, 42.0]",F,Diesel,Pays-de-la-Loire,"(40.0, 200.0]"


In [ ]:
dummies = pd.get_dummies(contracts["Power"], prefix="Power")

In [ ]:
contracts = contracts.join(dummies)

In [ ]:
dummies = pd.get_dummies(contracts["Brand"], prefix="Brand")

In [ ]:
contracts = contracts.join(dummies)



### 14.4.1 Negative Binomial Regression.




In [ ]:
# Using GLM 
from statsmodels.formula.api import glm
form = 'ClaimNb ~ DriverAge + CarAge + Density + Brand_F + Power_def + Power_gh + Gas'                
regnb2_glm = glm(formula = form, data = contracts ,offset = np.log(contracts.Exposure), family = sm.families.NegativeBinomial()).fit()

In [ ]:
print(regnb2_glm.summary2())

                                 Results: Generalized linear model
Model:                         GLM                         AIC:                       134745.8615  
Link Function:                 log                         BIC:                       -5251527.2116
Dependent Variable:            ClaimNb                     Log-Likelihood:            -67359.      
Date:                          2022-04-08 04:11            LL-Null:                   -68921.      
No. Observations:              413169                      Deviance:                  91233.       
Df Model:                      13                          Pearson chi2:              7.00e+05     
Df Residuals:                  413155                      Scale:                     1.0000       
Method:                        IRLS                                                                
---------------------------------------------------------------------------------------------------
                                 

In [ ]:
print("the over dispersion in the model is :",(regnb2_glm.pearson_chi2 / regnb2_glm.df_resid))

the over dispersion in the model is : 1.6933740169931542


In [ ]:
print("the residual deviance of the model is :",regnb2_glm.df_resid)

the residual deviance of the model is : 413155


In [ ]:
print("the degrees of freedom of the model is :",regnb2_glm.df_model)

the degrees of freedom of the model is : 13
